<a href="https://colab.research.google.com/github/JoanChege/JoanChege/blob/main/NLP2_%26_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio -q
!pip install sentence-transformers -q
!pip install nltk -q

import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [34]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import warnings
warnings.filterwarnings('ignore')

stop = stopwords.words('english')
print(len(stop))

179


In [ ]:
df = pd.read_csv('titles.csv')
df.sample(10)

In [ ]:
sample = df.sample(110000)
sample.head()

In [ ]:
sample = sample.reset_index(drop = True)
sample.head()

In [ ]:
#cleaning
sample['cleantitle'] = sample['title'].str.replace('[^\w\s]', '')
sample['cleantitle'] = sample['cleantitle'].str.replace('\d+', '')
sample['cleantitle'] = sample['cleantitle'].str.lower()

sample['cleantitle'] = sample['cleantitle'].apply(lambda item: " ".join(item for item in item.split()if item not in stop))
sample.head()

In [39]:
### feature extraction

model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [ ]:
features = model.encode(sample['title'])
sample['features'] = list(features)
sample.head()

In [41]:
print(sample['title'].iloc[46])
print(sample['features'].iloc[46].shape)
#print(sample['features'].iloc[0])

Proactive Algorithms for Scheduling with Probabilistic Durations.
(768,)


In [42]:
sample_query = sample.iloc[0]['title']
sample_query

'Constraint-Based Sentence Compression: An Integer Programming Approach.'

In [43]:
query_feature = model.encode([sample_query])

In [44]:
sample['similarity'] = sample['features'].apply(lambda x: cosine_similarity([x], query_feature)[0][0])
sample.head()

,title,cleantitle,features,similarity
0,Constraint-Based Sentence Compression: An Inte...,constraintbased sentence compression integer p...,"[-1.5604994, -0.5958466, -0.06118701, -0.51249...",1.000000
1,Extract Package Refactoring in ARIES.,extract package refactoring aries,"[-0.89603543, -0.2807616, -0.086970985, -0.419...",0.527305
2,A Frame of Mind: Using Statistical Models for ...,frame mind using statistical models detection ...,"[-0.09149999, -0.4401171, 0.06439174, -0.45548...",0.500987
3,StarrySky: A Practical System to Track Million...,starrysky practical system track millions high...,"[-0.74192446, -1.0004094, 0.82756627, 0.500512...",0.531209
4,Graph sketcher: extending illustration to quan...,graph sketcher extending illustration quantita...,"[-1.2392706, -0.111000694, 0.4541993, -0.54812...",0.581231


In [45]:
sorted_sample = sample.sort_values(by='similarity', ascending=False)
sorted_sample.tail()

,title,cleantitle,features,similarity
98043,On the weakest failure detector ever.,weakest failure detector ever,"[-1.080253, -0.9041112, 0.23381129, 0.51915187...",0.086099
4729,What you always wanted to know about agile met...,always wanted know agile methods dare ask,"[-0.8124281, 0.11575549, 0.43745005, 0.2083621...",0.076706
77532,Scott Is Always Simple.,scott always simple,"[-1.5633714, -0.78141797, -0.58502847, 0.15638...",0.070837
10860,The role of music in the lives of homeless you...,role music lives homeless young people seattle...,"[-0.0838376, -0.20004673, -0.01804788, -1.1827...",0.063980
75782,The Next Best Underwater View.,next best underwater view,"[-0.3536422, -0.2167202, 1.4583498, -0.2241654...",0.061130


In [46]:
def return_similar(query):
  query_feature = model.encode([query])
  sample['similarity'] = sample['features'].apply(lambda x: cosine_similarity([x], query_feature)[0][0])
  sorted_sample = sample.sort_values(by='similarity', ascending=False)
  result = sorted_sample['title'].head(10)
  return list(result)

In [47]:
x = str(input(" enter a query"))
print(return_similar(x))

 enter a querymaths
['Mathematics by machine.', 'Advanced notations in Mathematica.', 'The Ground Congruence for Chi Calculus.', 'Mathematical Theory Exploration.', 'The MP (Materialization Pattern) Model for Representing Math Educational Standards.', 'Data exchange with arithmetic operations.', 'Enhancing Mathematics Information Retrieval.', 'Aligator: A Mathematica Package for Invariant Generation (System Description).', 'Arithmetic Branching Programs with Memory.', 'The CoRe Calculus.']


In [49]:
import gradio as gr 

iface = gr.Interface(fn=return_similar, inputs = "text", outputs = "text")
iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`

Using Embedded Colab Mode (NEW). If you have issues, please use share=True and file an issue at https://github.com/gradio-app/gradio/
Note: opening the browser inspector may crash Embedded Colab Mode.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

(<gradio.routes.App at 0x7f92b83dd1d0>, 'http://127.0.0.1:7863/', None)

In [48]:
import gradio as gr 

iface = gr.Interface(fn=return_similar, inputs = "text", outputs = "text")
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://cc7b45115c602350.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7f92b83be810>,
 'http://127.0.0.1:7862/',
 'https://cc7b45115c602350.gradio.app')